In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import requests
import os
import csv

# 이미지를 저장할 디렉터리 확인 및 생성
image_dir = './images'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
data = []  # 데이터를 저장할 리스트
# Selenium 드라이버 설정
options = webdriver.ChromeOptions()
options.headless = True  # 브라우저 창을 띄우지 않는 헤드리스 모드
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 웹 페이지 로드
url = "https://pcmap.place.naver.com/restaurant/1235520676/menu/list"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기
time.sleep(1)  # 로딩 시간에 따라 조정이 필요할 수 있음

# 이미지가 포함된 div 태그를 찾음
img_tags = driver.find_elements(By.CLASS_NAME, 'K0PDV')

# 이미지 URL 및 음식 이름 추출
for index, img_tag in enumerate(img_tags):
    style_attr = img_tag.get_attribute('style')
    img_url_match = re.search(r"background-image: url\(\"(.*?)\"\)", style_attr)
    
    if img_url_match:
        img_url = img_url_match.group(1)
        print("찾은 이미지 URL:", img_url)

        # Find the corresponding food name for the current image
        food_name_element = img_tag.find_element(By.CLASS_NAME, 'place_blind')
        food_name = food_name_element.text.strip()
        print("찾은 음식 이름:", food_name)

        # 이미지 다운로드
        img_response = requests.get(img_url)
        img_response.raise_for_status()
        data.append({'FoodName': food_name, 'ImageURL': img_url})

        # 이미지 파일 저장
        img_extension = img_url.split('.')[-1].split('?')[0]  # 쿼리 파라미터 제거
        img_filename = f'{food_name}_{index}.{img_extension}'
        img_filepath = os.path.join(image_dir, img_filename)
        with open(img_filepath, 'wb') as img_file:
            img_file.write(img_response.content)
        print(f"이미지 저장됨: {img_filepath}")
    else:
        print("이미지 URL을 찾을 수 없습니다.")

# 드라이버 종료
driver.quit()

csv_file_path = 'Food_data.csv'
csv_columns = ['FoodName', 'ImageURL']

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
    writer.writeheader()
    for row in data:
        writer.writerow(row)

print("다운로드 및 CSV 파일 생성 완료.")


찾은 이미지 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230526_64%2F1685109309274AJhif_JPEG%2FF9F67227-5BB2-4C98-8AC4-AB6E2E79D2FA.jpeg
찾은 음식 이름: 
이미지 저장됨: ./images\_0.jpeg
찾은 이미지 URL: https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230603_288%2F1685804019331S56xK_JPEG%2F9B5B4481-EFEA-4328-B896-D04F5282689A.jpeg
찾은 음식 이름: 
이미지 저장됨: ./images\_1.jpeg
찾은 이미지 URL: https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230508_58%2F1683532471315q8iTU_JPEG%2FE44AED3A-D00B-43D4-837A-A267DF6E10E1.jpeg
찾은 음식 이름: 
이미지 저장됨: ./images\_2.jpeg
찾은 이미지 URL: https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230429_66%2F1682697744665pHPt3_JPEG%2F7D0A1705-BBD0-49D0-B5CD-54348939328F.jpeg
찾은 음식 이름: 업체
이미지 저장됨: ./images\업체_3.jpeg
찾은 이미지 URL: https://search.